### Pricing Model for Convertible Bond
This notebook provides several pricing models API:
+ **Black-Scholes model** for vanilla convertible bond pricing
+ **Binomial Tree pricing method** for American type convertible bond
+ **Monte Carlo simulation method** adoptable for convertible bond we interested in (with callability/putability/CoCo items, etc)

This notebook is imported in `Run Model.ipynb` to calculate daily model price for the samples in dataset.

*reference:
[Convertible Bond Pricing: A Monte Carlo Approach](https://web.bndes.gov.br/bib/jspui/bitstream/1408/7001/1/Mestrado_Leandro%20Amato%20Loriato_com%20termo_P.pdf)*

In [1]:
import math
import random
import numpy as np
import pandas as pd
import scipy.stats as st
import statsmodels.api as sm
from scipy.special import comb

#### Black-Scholes formula
$$ CB = \sum_{\tau \in \Omega_{coupon}}e^{-r\tau} + n(T)S_0 e^{-qT}N(d_1) + e^{-rT}\kappa N(1-N(d_2)) $$

In [2]:
def BSmodel(para):
    """
    Black-SCholes pricing model, only works for vanilla convertible bond
    para <dict>: convertible bond parameters
    return: pure bond price and convertible bond price
    """
    if para['coupon'] is None:
        coupon = 0
    else:
        coupon = np.sum([math.e ** (-t * para['cr']) * para['face_value'] * rate for t, rate in para['coupon'].items()])
    
    d1 = (np.log(para['conv_ratio'] * para['S'] / (para['redemption'] * para['face_value'])) + (para['r'] - para['q'] + para['sigma'] ** 2 / 2) * para['maturity']) / (para['sigma'] * np.sqrt(para['maturity']))
    d2 = d1 - para['sigma'] * np.sqrt(para['maturity'])
    bond = coupon + para['face_value'] * para['redemption'] * math.e ** (-para['cr'] * para['maturity'])
    cbond = coupon + \
            para['conv_ratio'] * para['S'] * math.e ** (-para['q'] * para['maturity']) * st.norm.cdf(d1) + \
            para['redemption'] * para['face_value'] * math.e ** (- para['cr'] * para['maturity']) * (1 - st.norm.cdf(d2))
    return bond, cbond

In [3]:
def TreeModel(para, step=1000, cbond_type='AM'):
    """
    Binomial Tree-based pricing model
    para <dict>: convertible bond parameters
    step <int>: total steps binomial tree grows
    cbond_type <string>: 'EU' or 'AM'(European type or American type)
    """
    delta_t = para['maturity'] / step
    u = math.e ** (para['sigma'] * np.sqrt(delta_t))
    d = 1/u
    p = (math.e ** ((para['r'] - para['q']) * delta_t) - d) / (u-d)
    S = para['S']
    
    tree = {}
    layer = 0
    t = 0
    
    # coupon calculation
        
    if para['coupon'] is None:
        discount_coupon = 0
        coupon_date = []
        
    else:
        coupon_date = list(para['coupon'].keys())
        discount_coupon = np.sum([math.e ** (-t * para['cr']) * para['face_value'] * rate for t, rate in para['coupon'].items()])
  
        # accrued interest
        ac_date = np.array([i for i in np.arange(0, para['maturity']+1e-8, para['maturity'] / step)])
        ac_coupon = np.array([0 for I in range(len(ac_date))])

        last_period = 0
        for period, rate in para['coupon'].items():
            select = (ac_date > last_period) & (ac_date <= period)
            ac_coupon = np.where(select, rate * para['face_value'] / select.sum(), ac_coupon)
            ac_coupon[select] = np.cumsum(ac_coupon[select])
            last_period = period
        
        ac_date = list(ac_date)
        ac_coupon = list(ac_coupon)

    # forward induction: grow the tree
    for s in range(step+1):
        t = s * delta_t
        layer += 1
        node = [S * u**i * d**(layer-1-i) for i in range(layer)]
        prob = [comb(layer-1,i) * p**i * (1-p)**(layer-1-i) for i in range(layer)]
        tree[layer] = {'node':node, 'prob':prob}
        
    # EU type return
    if cbond_type == 'EU':
        tree[layer]['node'] = [max(S * para['conv_ratio'], para['face_value'] * para['redemption']) for S in tree[layer]['node']]
        cbond = math.e **(-para['maturity'] * para['cr']) * np.dot(tree[layer]['node'], tree[layer]['prob']) + discount_coupon
        return cbond
    
    # backward induction: consider conversion/callability/putability for each step
    for l in range(layer, 0, -1):
        date = (l - 1) * delta_t
        idx = layer - 1
        
        # coupon
        if coupon_date and (date - delta_t < coupon_date[-1]) and (date >= coupon_date[-1]):
            cd = coupon_date.pop()
            coupon = para['coupon'][cd] * para['face_value']

            ac_coupon.pop()
            acc_coupon = 0
        else:
            coupon = 0
            acc_coupon = ac_coupon.pop()
#             acc_coupon = 0
        
        # conv_ratio
        if para['conv_ratio'] is not None and date >= para['conv_date'][0] and date <= para['conv_date'][1]:
            conv_ratio = para['conv_ratio']
        else:
            conv_ratio = 0
        
        # call
        if para['call'] is not None and date >= para['call_date'][0] and date <= para['call_date'][1]:
            if para['call'] == 100:
                call = para['call'] + acc_coupon
            else:
                call = para['call']
        else:
            call = np.inf
        
        # put
        if para['put'] is not None and date >= para['put_date'][0] and date <= para['put_date'][1]:
            if para['put'] == 100:
                put = para['put'] + acc_coupon
            else:
                put = para['put']
        else:
            put = 0
        
        # calculate convertible bond value
        if l == layer:
            tree[l]['node'] = [max(S * conv_ratio, para['face_value'] * para['redemption']) + coupon
                               for S in tree[l]['node']]
        else:
            tree[l]['node'] = [max(
                min(math.e ** (- para['cr'] * delta_t) * (p * tree[l+1]['node'][n+1] + (1-p)*tree[l+1]['node'][n]), call),
                put,
                tree[l]['node'][n] * conv_ratio) + coupon for n in range(len(tree[l]['node']))]
    
    if len(ac_coupon) != 0:
        print('accrue coupon: {}'.format(ac_coupon))
    return tree[1]['node'][0]  

In [4]:
def MonteCarlo(para, trial=1000, step=100, cbond_type='EU', forward='euler', backward='LSMC'):
    """
    MonteCarlo simulation method for convertible bond pricing
    para <dict>: convertible bond parameters
    trial <int>: total number of simulation trials
    step <int>: total number of step for each simulation trial
    cbond_type <string>: 'EU' or 'AM'
    forward <string>: 'euler' or 'milstein', different methods used for stock price forward simulation
    backward <string>: 'LSMC' or 'HMC', different methods used to calculate backward discounted value
    """
    
    delta_t = para['maturity'] / step
    
    # coupon calculation
    if para['coupon'] is None:
        discount_coupon = 0
        coupon_date = []
    else:
        coupon_date = list(para['coupon'].keys())
        discount_coupon = np.sum([math.e ** (-t * para['cr']) * para['face_value'] * rate for t, rate in para['coupon'].items()])
        
        # accrued interest
        ac_date = np.array([i for i in np.arange(0, para['maturity']+1e-8, para['maturity'] / step)])
        ac_coupon = np.array([0 for I in range(len(ac_date))])

        last_period = 0
        for period, rate in para['coupon'].items():
            select = (ac_date > last_period) & (ac_date <= period)
            if select.sum() != 0:
                ac_coupon = np.where(select, rate * para['face_value'] / select.sum(), ac_coupon)
            ac_coupon[select] = np.cumsum(ac_coupon[select])
            last_period = period
        
        ac_date = list(ac_date)
        ac_coupon = list(ac_coupon)
        
    mc = []
    
    # forward induction: simulate time-series samples for future stock price
    for i in range(trial):
        date = 0
        W = 0
        S = para['S']
        series = [S]
        for j in range(step):
            date += (j+1) * delta_t
            z = np.random.randn(1)[0]
            W += z * np.sqrt(delta_t)
            if forward == 'euler':
                S += S * ((para['r'] - para['q']) * delta_t + para['sigma'] * z * np.sqrt(delta_t))
            else:
                S += S * ((para['r'] - para['q']) * delta_t + para['sigma'] * z * np.sqrt(delta_t) + 1/2 * para['sigma']**2 * (z**2 - 1) * delta_t)
            series.append(S)
        
        mc.append(series) 
        
    # EU type pricing
    if cbond_type == 'EU':
        cbond = np.mean([math.e ** (-para['maturity'] * para['cr']) * max(series[-1] * para['conv_ratio'], para['face_value'] * para['redemption'])
                for series in mc]) + discount_coupon
        return cbond
    
    # AM type pricing
    stock = pd.DataFrame(mc) # shape: trial * (step + 1)
    cbond = pd.DataFrame(mc)
    
    # construct helper array 
    if para['conv_adj'] is not None:
        win = para['conv_adj'][0]
        stock_conv = stock.rolling(win, axis=1).mean()
        stock_conv_ratio = np.where((stock_conv >= para['conv_adj'][1] * para['S']).fillna(0).cumsum(axis=1) > 0, 
                                    para['conv_adj'][2], para['conv_ratio'])
    
    if para['call'] is not None and para['call_soft'] is not None:
        win = para['call_soft'][0]
        stock_call_count = (stock >= para['call_soft'][2] * para['conv_price']).astype('int64').rolling(win, axis=1).sum()
        stock_call = (stock_call_count >= para['call_soft'][1])
        stock_call_soft = np.where(stock_call.fillna(0).cumsum(axis=1) > 0, para['call'], np.inf)       
            
    if para['put'] is not None and para['put_soft'] is not None:
        win = para['put_soft'][0]
        stock_put_count = (stock <= para['put_soft'][2] * para['conv_price']).astype('int64').rolling(win, axis=1).sum()
        stock_put = (stock_put_count >= para['put_soft'][1])
        stock_put_soft = np.where(stock_put.fillna(0).cumsum(axis=1) > 0, para['put'], 0)  

    # backward induction: consider conversion/callability/putability for each step
    for i in range(step + 1):
        date = para['maturity'] - i * delta_t
        idx = step - i 
        
        # coupon
        if coupon_date and (date - delta_t < coupon_date[-1]) and (date >= coupon_date[-1]):
            cd = coupon_date.pop()
            coupon = para['coupon'][cd] * para['face_value']
            ac_coupon.pop()
            acc_coupon = 0
        else:
            coupon = 0
            acc_coupon = ac_coupon.pop()
#             acc_coupon = 0
            
        # conv_ratio
        if para['conv_ratio'] is not None and date >= para['conv_date'][0] and date <= para['conv_date'][1]:
            if para['conv_adj'] is not None:
                conv_ratio = stock_conv_ratio[:, idx]  # contigent conversion
            else:
                conv_ratio = para['conv_ratio']
        else:
            conv_ratio = 0
        
        # call
        if para['call'] is not None and date >= para['call_date'][0] and date <= para['call_date'][1]:
            if para['call'] == 100:
                add = acc_coupon
            else:
                add = 0
                
            if para['call_soft'] is not None:
                call = stock_call_soft[:, idx] + add # soft call
            else:
                call = para['call'] + add
        else:
            call = np.inf
        
        # put
        if para['put'] is not None and date >= para['put_date'][0] and date <= para['put_date'][1]:
            if para['put'] == 100:
                add = acc_coupon
            else:
                add = 0
                
            if para['put_soft'] is not None:
                put = stock_put_soft[:, idx] + add   # soft put
            else:
                put = para['put'] + add
        else:
            put = 0
        
        # special case: at the end of simulation
        if date == para['maturity']:
            cbond.iloc[:, idx] = np.maximum(stock.iloc[:, idx].values * conv_ratio, para['face_value'] * para['redemption']) + coupon
                                        
        else:
            if backward == 'LSMC':
                X = Hermite(stock.iloc[:, idx])
            else:
                X1 = Hermite(stock.iloc[:, idx])
                X2 = partialHermite(stock.iloc[:, idx]).multiply(
                    (math.e ** (-delta_t * para['cr']) * stock.iloc[:, idx+1] - stock.iloc[:, idx]) ** 2, axis='index')
                X = pd.concat([X1, X2], axis=1)
                X.columns = [i for i in range(len(X.columns))]
            y = math.e ** (-delta_t * para['cr']) * (cbond.iloc[:, idx+1] .values)
            model = sm.OLS(y, X)
            results = model.fit()
            y_hat = np.dot(X, results.params)
            
            conv_value = stock.iloc[:, idx].values * conv_ratio
            value = np.maximum(np.maximum(np.minimum(y_hat, call), conv_value), put)
            cbond.iloc[:, idx] = np.where(value != y_hat, value, y) + coupon
    
    if len(ac_coupon) != 0: # algorithm check, ac_coupon should be empty here
        print('Warning! Not empty accrue coupon: {}'.format(accrue_coupon))
    return cbond[0].mean()


# helper funcion used in MonteCarlo 
def partialHermite(x):
    """
    helper function
    calculate partial Hermite polynomials, used when forward='milstein'
    """
    H = np.array([
        np.array([2] * len(x)),
        8 * x,
        8*3 * x**2 - 12,
        16*4 * x**3 - 48*2 * x,
        32*5 * x**4 - 160*3 * x**2 + 120])
    return pd.DataFrame(H).T

def Hermite(x):
    """
    helper function
    calculate Hermite polynomials, used in forward simulation
    """
    H = np.array([np.array([1] * len(x)),
        2 * x,
        4 * x**2 - 2,
        8 * x**3 - 12 * x,
        16 * x**4 - 48 * x**2 + 12,
        32 * x**5 - 160 * x**3 + 120*x])
    return pd.DataFrame(H).T

#### sample para

In [5]:
para = {
    # bond
    'maturity': 2,
    'face_value': 100,
    'coupon': dict(zip([0.5, 1, 1.5, 2], [0.05, 0.05, 0.05, 0.05])),
    # rdmpt
    'redemption': 1,
    # conv
    'conv_ratio': 1,
    'conv_price': 100, 
    'conv_date':(0, 2),
    'conv_adj':None,
    # call
    'call':110,
    'call_date':(0, 2),
    'call_soft':(20, 20, 1.1),
    # put
    'put': 98,
    'put_date':(0, 2),
    'put_soft':None,
    # stock
    'r': 0.05,
    'S': 100,
    'sigma': 0.4,
    'q': 0.1,
    # credit spread
    'cr':0.05,
}

#### sample price calculated by models

In [6]:
BSmodel(para)
# return: pure bond price and convertible bond price

(109.27934300806373, 124.45706924117123)

In [10]:
TreeModel(para, step=1000, cbond_type='EU')

124.452210536628

In [8]:
MonteCarlo(para, trial=10000, step=100, cbond_type='AM', forward='milstein', backward='LSMC')

122.86292059179927

In [9]:
MonteCarlo(para, trial=10000, step=100, cbond_type='AM', forward='euler', backward='LSMC')

122.836571793141